## Setup

In [1]:
# SWA weights
!/home/jupyter/.local/bin/gdown --id 1-0mtggsTMLDmbT7cmxdIBH25eAieTyGb
!/home/jupyter/.local/bin/gdown --id 1--EvBdSAnNaBtKozkMuja--lq8erN0uC

# MA weights
!/home/jupyter/.local/bin/gdown --id 1M0JwphtV6W1NM6V2Qv_VMTdjWdeAz6Sx
!/home/jupyter/.local/bin/gdown --id 1pW2hx6fkQGH8fXFfh8Fn8xPADuYBy5mG

Downloading...
From: https://drive.google.com/uc?id=1-0mtggsTMLDmbT7cmxdIBH25eAieTyGb
To: /home/jupyter/teacher_model_swa.h5
94.6MB [00:01, 84.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1--EvBdSAnNaBtKozkMuja--lq8erN0uC
To: /home/jupyter/student_noisy_swa.h5
94.6MB [00:01, 76.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1M0JwphtV6W1NM6V2Qv_VMTdjWdeAz6Sx
To: /home/jupyter/teacher_model_ma.h5
94.6MB [00:00, 107MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1pW2hx6fkQGH8fXFfh8Fn8xPADuYBy5mG
To: /home/jupyter/student_noisy_ma.h5
94.6MB [00:01, 76.7MB/s]


In [13]:
from tensorflow.keras import layers
import tensorflow as tf

import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from tqdm import tqdm
import numpy as np

## Define Hyperparameters

In [15]:
AUTO = tf.data.AUTOTUNE
DATASET_NAME = "cifar10_corrupted"
BATCH_SIZE = 64
IMAGE_SIZE = 72

In [6]:
VERSIONS = [
    "brightness_5",
    "contrast_5",
    "defocus_blur_5",
    "elastic_5",
    "fog_5",
    "frost_5",
    "frosted_glass_blur_5",
    "gaussian_blur_5",
    "gaussian_noise_5",
    "impulse_noise_5",
    "jpeg_compression_5",
    "motion_blur_5",
    "pixelate_5",
    "saturate_5",
    "shot_noise_5",
    "snow_5",
    "spatter_5",
    "speckle_noise_5",
    "zoom_blur_5"
]

print(f"Total sub-versions of the CIFAR10-C dataset: {len(VERSIONS)}")

Total sub-versions of the CIFAR10-C dataset: 19


## Utilities

In [5]:
def prepare_dataset(ds):
    ds = (ds
          .batch(BATCH_SIZE)
          .map(lambda x, y: (tf.image.resize(x, (IMAGE_SIZE, IMAGE_SIZE)), y), 
               num_parallel_calls=AUTO)
          .prefetch(AUTO)
    )
    return ds

In [11]:
def get_training_model(num_classes=10):
    resnet50_v2 = tf.keras.applications.ResNet50V2(
        weights=None,
        include_top=False,
        input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
    )
    model = tf.keras.Sequential(
        [
            layers.Input((IMAGE_SIZE, IMAGE_SIZE, 3)),
            layers.experimental.preprocessing.Rescaling(scale=1.0 / 127.5, offset=-1),
            resnet50_v2,
            layers.GlobalAveragePooling2D(),
            layers.Dense(num_classes)
        ]
    )
    return model

In [9]:
def evaluate_model(model):
    acc_dict = {}
    for version in tqdm(VERSIONS):
        print(f"Processing {version}")
        dataset_fullname = DATASET_NAME + "/" + version
        loaded_ds = tfds.load(
            dataset_fullname,
            split="test",
            as_supervised=True
        )
        loaded_ds = prepare_dataset(loaded_ds)
        _, acc = model.evaluate(loaded_ds, verbose=0)
        print(f"Test accuracy on {version}: {acc*100}%")
        acc_dict[version] = acc*100
        
    return acc_dict, np.mean(list(acc_dict.values()))

## Evaluation

In [16]:
# Evaluate teacher model trained with SWA
teacher_model_swa = get_training_model()
teacher_model_swa.load_weights("teacher_model_swa.h5")
teacher_model_swa.compile(loss="sparse_categorical_crossentropy",
                         metrics=["accuracy"])
acc_dict, mean_top_1 = evaluate_model(teacher_model_swa)
print(mean_top_1)

  0%|          | 0/19 [00:00<?, ?it/s]

Processing brightness_5


  5%|▌         | 1/19 [00:17<05:20, 17.80s/it]

Test accuracy on brightness_5: 75.60999989509583%
Processing contrast_5
Dataset cifar10_corrupted downloaded and prepared to /home/jupyter/tensorflow_datasets/cifar10_corrupted/contrast_5/1.0.0. Subsequent calls will reuse this data.


 11%|█         | 2/19 [00:28<03:53, 13.74s/it]

...

Test accuracy on spatter_5: 63.88000249862671%
Processing speckle_noise_5
Dataset cifar10_corrupted downloaded and prepared to /home/jupyter/tensorflow_datasets/cifar10_corrupted/speckle_noise_5/1.0.0. Subsequent calls will reuse this data.


 95%|█████████▍| 18/19 [02:59<00:09,  9.37s/it]

Test accuracy on speckle_noise_5: 46.880000829696655%
Processing zoom_blur_5
Dataset cifar10_corrupted downloaded and prepared to /home/jupyter/tensorflow_datasets/cifar10_corrupted/zoom_blur_5/1.0.0. Subsequent calls will reuse this data.


100%|██████████| 19/19 [03:09<00:00,  9.97s/it]

Test accuracy on zoom_blur_5: 78.329998254776%
59.21105271891544


In [17]:
# Evaluate the corresponding student model
student_noisy_swa = get_training_model()
student_noisy_swa.load_weights("student_noisy_swa.h5")
student_noisy_swa.compile(loss="sparse_categorical_crossentropy",
                         metrics=["accuracy"])
acc_dict, mean_top_1 = evaluate_model(student_noisy_swa)
print(mean_top_1)

  0%|          | 0/19 [00:00<?, ?it/s]

Processing brightness_5


  5%|▌         | 1/19 [00:03<01:03,  3.52s/it]

Test accuracy on brightness_5: 80.91999888420105%
Processing contrast_5


 11%|█         | 2/19 [00:05<00:45,  2.70s/it]

...

Test accuracy on spatter_5: 66.00000262260437%
Processing speckle_noise_5


 95%|█████████▍| 18/19 [00:39<00:02,  2.13s/it]

Test accuracy on speckle_noise_5: 48.85999858379364%
Processing zoom_blur_5


100%|██████████| 19/19 [00:41<00:00,  2.21s/it]

Test accuracy on zoom_blur_5: 77.31000185012817%
60.10999993274086


In [18]:
# Evaluate teacher model trained with MA
teacher_model_ma = get_training_model()
teacher_model_ma.load_weights("teacher_model_ma.h5")
teacher_model_ma.compile(loss="sparse_categorical_crossentropy",
                         metrics=["accuracy"])
acc_dict, mean_top_1 = evaluate_model(teacher_model_ma)
print(mean_top_1)

  0%|          | 0/19 [00:00<?, ?it/s]

Processing brightness_5


  5%|▌         | 1/19 [00:03<00:58,  3.25s/it]

Test accuracy on brightness_5: 72.39000201225281%
Processing contrast_5


 11%|█         | 2/19 [00:05<00:43,  2.58s/it]

...

Test accuracy on spatter_5: 61.169999837875366%
Processing speckle_noise_5


 95%|█████████▍| 18/19 [00:39<00:02,  2.11s/it]

Test accuracy on speckle_noise_5: 48.2699990272522%
Processing zoom_blur_5


100%|██████████| 19/19 [00:41<00:00,  2.17s/it]

Test accuracy on zoom_blur_5: 76.78999900817871%
58.81736843209518


In [19]:
# Evaluate the corresponding student model
student_noisy_ma = get_training_model()
student_noisy_ma.load_weights("student_noisy_ma.h5")
student_noisy_ma.compile(loss="sparse_categorical_crossentropy",
                         metrics=["accuracy"])
acc_dict, mean_top_1 = evaluate_model(student_noisy_ma)
print(mean_top_1)

  0%|          | 0/19 [00:00<?, ?it/s]

Processing brightness_5


  5%|▌         | 1/19 [00:03<00:59,  3.32s/it]

Test accuracy on brightness_5: 78.72999906539917%
Processing contrast_5


 11%|█         | 2/19 [00:05<00:44,  2.62s/it]

...

Test accuracy on spatter_5: 66.25000238418579%
Processing speckle_noise_5


 95%|█████████▍| 18/19 [00:40<00:02,  2.15s/it]

Test accuracy on speckle_noise_5: 48.42999875545502%
Processing zoom_blur_5


100%|██████████| 19/19 [00:42<00:00,  2.22s/it]

Test accuracy on zoom_blur_5: 72.94999957084656%
57.67210478845396
